In [1]:
import pandas as pd
import os
import warnings

warnings.filterwarnings('ignore')

In [3]:
df = pd.read_excel('用户偏好分析案例数据.xlsx')
df.head()

,品牌名称,用户ID,付款日期,订单状态,实付金额,邮费,省份,城市,购买数量
0,阿粥（小z),uid00324123,2023-04-18,交易成功,22.32,0,北京,北京市,1
1,阿粥（小z),uid00324124,2023-02-17,交易成功,87.00,0,上海,上海市,1
2,阿粥（小z),uid00324125,2023-04-18,交易成功,97.66,0,福建省,福州市,2
3,阿粥（小z),uid00324126,2023-01-11,交易成功,37.23,0,河南省,安阳市,3
4,阿粥（小z),uid00324126,2023-02-18,交易成功,29.50,0,河南省,安阳市,2


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28832 entries, 0 to 28831
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   品牌名称    28832 non-null  object        
 1   用户ID    28832 non-null  object        
 2   付款日期    28832 non-null  datetime64[ns]
 3   订单状态    28832 non-null  object        
 4   实付金额    28832 non-null  float64       
 5   邮费      28832 non-null  int64         
 6   省份      28832 non-null  object        
 7   城市      28832 non-null  object        
 8   购买数量    28832 non-null  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(5)
memory usage: 2.0+ MB


In [5]:
#set avg amount > 50 as high quality user
gp_user = df.groupby('用户ID')['实付金额'].mean().reset_index()
gp_user.columns = ['用户ID','平均每次支付金额']
def if_high(x):
    if x > 50:
        return '高客单价'
    else:
        return '低客单价'

gp_user['客单价类别'] = gp_user['平均每次支付金额'].apply(if_high)

#add city
#drop duplicate
df_dup = df.loc[df.duplicated('用户ID') == False,:]
df_merge = pd.merge(gp_user,df_dup,left_on = '用户ID',right_on = '用户ID',how = 'left')
df_merge = df_merge[['用户ID','平均每次支付金额','客单价类别','省份','城市']]

df_merge.head()

,用户ID,平均每次支付金额,客单价类别,省份,城市
0,uid00324123,22.320,低客单价,北京,北京市
1,uid00324124,87.000,高客单价,上海,上海市
2,uid00324125,97.660,高客单价,福建省,福州市
3,uid00324126,33.365,低客单价,河南省,安阳市
4,uid00324127,42.500,低客单价,浙江省,衢州市


$TGI = \frac{目标群体中具有某一特征的比例}{总体中具有相同特征的比例} * 100\%$

In [6]:
#calculate TGI
df_merge = df_merge[['用户ID','客单价类别','省份','城市']]
result = pd.pivot_table(df_merge,index =['省份','城市'],columns = '客单价类别',aggfunc = 'count')

tgi = pd.merge(result['用户ID']['高客单价'].reset_index(),
               result['用户ID']['低客单价'].reset_index(),
               left_on = ['省份','城市'],right_on = ['省份','城市'],how = 'inner')

tgi['总人数'] = tgi['高客单价'] + tgi['低客单价']
tgi['高客单价占比'] = tgi['高客单价'] / tgi['总人数']

tgi.head()

,省份,城市,高客单价,低客单价,总人数,高客单价占比
0,上海,上海市,2374.0,2818.0,5192.0,0.457242
1,云南省,临沧市,2.0,3.0,5.0,0.400000
2,云南省,丽江市,3.0,1.0,4.0,0.750000
3,云南省,保山市,2.0,6.0,8.0,0.250000
4,云南省,大理白族自治州,8.0,9.0,17.0,0.470588


In [7]:
tgi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 346 entries, 0 to 345
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   省份      346 non-null    object 
 1   城市      346 non-null    object 
 2   高客单价    332 non-null    float64
 3   低客单价    329 non-null    float64
 4   总人数     315 non-null    float64
 5   高客单价占比  315 non-null    float64
dtypes: float64(4), object(2)
memory usage: 16.3+ KB


In [8]:
tgi = tgi.dropna()
total_percentage = tgi['高客单价'].sum() / tgi['总人数'].sum()

tgi['高客单价TGI'] = tgi['高客单价占比'] / total_percentage * 100
tgi = tgi.sort_values('高客单价TGI',ascending = False)

In [9]:
#attention: low number of people will easily cause high TGI, so we should set a threshhold to filter   
tgi.loc[tgi['总人数'] > tgi['总人数'].mean(),:].head(10)

,省份,城市,高客单价,低客单价,总人数,高客单价占比,高客单价TGI
287,福建省,福州市,145.0,135.0,280.0,0.517857,124.699807
124,广东省,珠海市,49.0,52.0,101.0,0.485149,116.823582
27,北京,北京市,1203.0,1298.0,2501.0,0.481008,115.826450
283,福建省,厦门市,105.0,118.0,223.0,0.470852,113.380991
111,广东省,佛山市,118.0,135.0,253.0,0.466403,112.309708
173,江西省,南昌市,63.0,73.0,136.0,0.463235,111.546887
46,四川省,成都市,287.0,334.0,621.0,0.462158,111.287429
0,上海,上海市,2374.0,2818.0,5192.0,0.457242,110.103682
164,江苏省,无锡市,135.0,162.0,297.0,0.454545,109.454376
120,广东省,深圳市,438.0,528.0,966.0,0.453416,109.182440
